# Multi-Prompt Router Chains

This is based on an article from medium - https://medium.com/@princekrampah/langchain-routers-a6ad9a4f2ed9. The goal is to change the prompt and the type of persona used to answer a question based on the type of question asked. This gets into the concept of routing logic in Langchain, and a good walkthrough of different methods can be found here: https://python.langchain.com/v0.1/docs/expression_language/how_to/routing/


In [30]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# LLM Router chains
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
import openai_llm

In [32]:
# get the llm
llm = model = openai_llm.get_model()

data_scientist = """You are a Data Scientist with expertise in Deep Learning. Please mention who you are when answering the question.
          
Below is a question from a colleague:
{input}\
"""
data_analyst = """You are a Data Analyst with expertise in Business Intelligence and developing dashboards.Please mention who you are when answering the question.
          

Below is a question from a colleague:
{input}\
"""
application_developer = """You're a full stack application developer with extensive experience in web development. Please mention who you are when answering the question.
          
Below is a question from a colleague:
{input}\
"""

prompt_infos = [
    {
        "name": "Data Scientist",
        "description": "Good for answering questions regarding AI, Machine Learning and Deep Learning.",
        "prompt_template": data_scientist,
    },
    {
        "name": "Data Analyst",
        "description": "Good for answering questions regarding how to build business intelligence capabilities like dashboards and reports.",
        "prompt_template": data_analyst,
    },
    {
        "name": "Application Developer",
        "description": "Good for answering programming questions, and questions related to application development",
        "prompt_template": application_developer,
    },

]

# map destination chains
destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt_template = prompt_info["prompt_template"]
    # create the prompt
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # create the chain using the prompt and the LLM
    chain = LLMChain(llm=llm, prompt=prompt)
    # for each prompt create a chain and add to destation chains
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")

# Creating LLMRouterChain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# creating the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Multiple Prompt Chain
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

# test it out
print(chain.run("What is an outer join?"))
print(chain.run("How would I help a business customer create KPIs related to forecasting results?"))
print(chain.run("What are some good forecasting models that I could easily translate into a KPI?"))
print(chain.run("How would I build a chatbot?"))
print(chain.run("What is OOP?"))



> Entering new MultiPromptChain chain...
Data Analyst: {'input': 'What is an outer join?'}
> Finished chain.
An outer join is a type of join operation in relational databases that combines records from two or more tables, even if there is no matching record in one of the tables. It includes all the records from one table and matches them with the corresponding records from another table, and if no match is found, it includes NULL values in the result set.

For example, if we have two tables, Table A and Table B, and we perform a left outer join, it will include all the records from Table A and match them with the corresponding records from Table B. If there is no match, it will display NULL values for the columns from Table B.

I hope this explanation helps! Let me know if you have any further questions.



> Entering new MultiPromptChain chain...
Data Analyst: {'input': 'How can I help a business customer create KPIs related to forecasting results?'}
> Finished chain.
As a Data Anal